# [Advent of Code 2019: Day 10](https://adventofcode.com/2019/day/10)

<h2>--- Day 10: Monitoring Station ---</h2>
<p>You fly into the asteroid belt and reach the Ceres monitoring station. The Elves here have an emergency: they're having trouble tracking all of the asteroids and can't be sure they're safe.</p>
<p>The Elves would like to build a new monitoring station in a nearby area of space; they hand you a map of all of the asteroids in that region (your puzzle input).</p>
<p>The map indicates whether each position is empty (<code>.</code>) or contains an asteroid (<code>#</code>). The asteroids are much smaller than they appear on the map, and every asteroid is exactly in the center of its marked position. The asteroids can
    be described with <code>X,Y</code> coordinates where <code>X</code> is the distance from the left edge and <code>Y</code> is the distance from the top edge (so the top-left corner is <code>0,0</code> and the position immediately to its right is <code>1,0</code>).</p>
<p>Your job is to figure out which asteroid would be the best place to build a <em>new monitoring station</em>. A monitoring station can <em>detect</em> any asteroid to which it has <em>direct line of sight</em> - that is, there cannot be another asteroid
    <em>exactly</em> between them. This line of sight can be at any angle, not just lines aligned to the grid or <span title="The Elves on Ceres are clearly not concerned with honor.">diagonally</span>. The <em>best</em> location is the asteroid that
    can <em>detect</em> the largest number of other asteroids.</p>
<p>For example, consider the following map:</p>
<pre><code>.#..#
.....
#####
....#
...<em>#</em>#
</code></pre>
<p>The best location for a new monitoring station on this map is the highlighted asteroid at <code>3,4</code> because it can detect <code>8</code> asteroids, more than any other location. (The only asteroid it cannot detect is the one at <code>1,0</code>;
    its view of this asteroid is blocked by the asteroid at <code>2,2</code>.) All other asteroids are worse locations; they can detect <code>7</code> or fewer other asteroids. Here is the number of other asteroids a monitoring station on each asteroid
    could detect:</p>
<pre><code>.7..7
.....
67775
....7
...87
</code></pre>
<p>Here is an asteroid (<code>#</code>) and some examples of the ways its line of sight might be blocked. If there were another asteroid at the location of a capital letter, the locations marked with the corresponding lowercase letter would be blocked and
    could not be detected:</p>
<pre><code>#.........
...A......
...B..a...
.EDCG....a
..F.c.b...
.....c....
..efd.c.gb
.......c..
....f...c.
...e..d..c
</code></pre>
<p>Here are some larger examples:</p>
<ul>
    <li>
        <p>Best is <code>5,8</code> with <code>33</code> other asteroids detected:</p>
        <pre><code>......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...<em>#</em>..#.
.#....####
</code></pre>
    </li>
    <li>
        <p>Best is <code>1,2</code> with <code>35</code> other asteroids detected:</p>
        <pre><code>#.#...#.#.
.###....#.
.<em>#</em>....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
</code></pre>
    </li>
    <li>
        <p>Best is <code>6,3</code> with <code>41</code> other asteroids detected:</p>
        <pre><code>.#..#..###
####.###.#
....###.#.
..###.<em>#</em>#.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
</code></pre>
    </li>
    <li>
        <p>Best is <code>11,13</code> with <code>210</code> other asteroids detected:</p>
        <pre><code>.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.####<em>#</em>#####...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
</code></pre>
    </li>
</ul>
<p>Find the best location for a new monitoring station. <em>How many other asteroids can be detected from that location?</em></p>

In [1]:
import os
import unittest
import numpy as np
from operator import itemgetter
from fractions import Fraction

class AstroidMap():
    
    EMPTY = '.'
    ASTROID = '#'
    NEWLINE = "\n"
    
    MAX_TIMES_OFF_GRID = 4
    
    def __init__(self, map_data = []):
        if map_data:
            self.load_data(map_data)
        else:
            self.data = []
        
        self.laser_coord = None
        
    def load_data(self, map_data):
        x = 0
        y = 0
        self.data = []
        for point in map_data:
            if point is self.ASTROID:
                self.data.append((x,y))
                
            if point is self.NEWLINE:
                y += 1
                x = 0
            else:
                x +=1
        
        self.width = x + 1
        self.height = y + 1

    def get_slope(self, coord1, coord2):
        x1, y1 = coord1
        x2, y2 = coord2
        return((x1 - x2, y1 - y2))
    
    def incriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x+xstep, y+ystep)
    
    def decriment_coord(self, coord, step):
        x,y = coord
        xstep, ystep = step
        return (x-xstep, y-ystep)
    
    def get_step(self, coord):
        x, y = coord
        if x is 0:
            xcd = 0
            ysign = y / abs(y)
            ycd = ysign
            return (xcd, ycd)
        elif y is 0:
            xsign = x / abs(x)
            xcd = xsign
            ycd = 0
            return (xcd, ycd)
        else:
            xsign = x / abs(x)
            ysign = y / abs(y)
            cf = Fraction(x, y)
            xcd = abs(cf.numerator) * xsign
            ycd = cf.denominator * ysign
            
            return (xcd, ycd)
        
    def get_step_to(self, coord1, coord2):
        """ Returns a step for coordinate 2 that is in the direction of coordinate 1 """
        if coord1 is coord2:
            return (0,0)
        
        x1, y1 = coord1
        x2, y2 = coord2        
        dist = self.get_slope(coord1, coord2)
        xstep, ystep = self.get_step(dist)
            
        return (xstep, ystep)
    
    def is_valid(self, coord):
        x,y = coord
        if x < 0 or y < 0:
            return False
        elif x >= self.width or y >= self.height:
            return False
        else:
            return True
        
    def is_visible(self, coord1, coord2):
        """ Is coordinate 2 visible from coordinate 1 """
        if coord1 == coord2:
            return False
        
        coord2_step = self.get_step_to(coord1, coord2)
        check_coord = self.incriment_coord(coord2, coord2_step)
        while check_coord != coord1:
            if not self.is_valid(check_coord):
                raise Exception(f'Coordinate {check_coord} is off grid')
            if check_coord in self.data:
                return False
            check_coord = self.incriment_coord(check_coord, coord2_step)
        return True
    
    def count_visiblity(self, coord):
        """ Counts all visible astroids for given coordinate """
        
        count = 0
        for check_coord in self.data:
            if self.is_visible(coord, check_coord):
                count += 1
        return count
    
    def get_coord_with_best_visiblity(self):
        visiblity_list = []
        for coord in self.data:
            visiblity_list.append(self.count_visiblity(coord))
        most_visiblity = max(visiblity_list)
        self.laser_coord = self.data[visiblity_list.index(most_visiblity)]
        
        return self.laser_coord, most_visiblity
    
    def get_firing_angle_dist(self, target):
        sx, sy = self.get_slope(target, self.laser_coord)
        angle = np.arctan2(sx, -sy)
        if angle < 0:
            angle += 2 * np.pi
        distance = round(np.hypot(sx, sy),2)
        return angle, distance
    
    def run_laser(self, verbose = False):
        
        astroid_info = {}
        for coord in self.data:
            if coord is not self.laser_coord:
                angle, dist = self.get_firing_angle_dist(coord)
                if angle not in astroid_info:
                    astroid_info[angle] = []
                astroid_info[angle].append({'dist': dist, 'coord':coord})
                astroid_info[angle].sort(key=itemgetter('dist'), reverse=True)
    
        targets_destroyed = []
        while astroid_info:
            for angle in sorted(astroid_info.keys()):
                if verbose:
                    print(f'angle: {angle}, astroids: {astroid_info[angle]}')
                astroid = astroid_info[angle].pop()
                targets_destroyed.append(astroid['coord'])
                if verbose:
                    print(targets_destroyed[-1])
                if not astroid_info[angle]:
                    del astroid_info[angle]
        
        return targets_destroyed

class TestBasic(unittest.TestCase):
    
    def test_load_data(self):
        map_data = ".#..#\n.....\n#####\n....#\n...##"
        ans = [(1,0),(4,0),(0,2),(1,2),(2,2),(3,2),(4,2),(4,3),(3,4),(4,4)]
        am = AstroidMap(map_data)
        self.assertEqual(am.data ,ans)
        
    def test_get_step_to(self):
        data = [((1,0),(4,0),(-1,0))]
        am = AstroidMap()
        for coord1, coord2, ans in data:
            self.assertEqual(am.get_step_to(coord1, coord2), ans)

    def test_check_count_visiblity(self):
        map_data = ".#..#\n.....\n#####\n....#\n...##"
        data = [((1,0),7),((4,0),7),((0,2),6),((1,2),7),((2,2),7),((3,2),7),((4,2),5),((4,3),7),((3,4),8),((4,4),7)]
        am = AstroidMap(map_data)
        for coord, ans in data:
            self.assertEqual(am.count_visiblity(coord) ,ans)
            
    def test_run_laser(self):
        map_data = '.#..##.###...#######\n##.############..##.\n.#.######.########.#\n.###.#######.####.#.\n#####.##.#.##.###.##\n..#####..#.#########\n####################\n#.####....###.#.#.##\n##.#################\n#####.##.###..####..\n..######..##.#######\n####.##.####...##..#\n.#####..#.######.###\n##...#.##########...\n#.##########.#######\n.####.#.###.###.#.##\n....##.##.###..#####\n.#.#.###########.###\n#.#.#.#####.####.###\n###.##.####.##.#..##'
        test_data = [(0,(11,12)),(1,(12,1)),(2,(12,2)),(9,(12,8)),(19,(16,0)),(49,(16,9)),(99,(10,16)),(198,(9,6)),(199,(8,2)),(200,(10,9)),(298,(11,1))]
        am = AstroidMap(map_data)
        am.get_coord_with_best_visiblity()
        self.assertEqual(am.laser_coord,(11,13))
        targets_destroyed = am.run_laser(verbose=True)
        for index, ans in test_data:
            self.assertEqual(targets_destroyed[index],ans)
        
        
unittest.main(argv=[""], exit=False)

....

angle: 0.0, astroids: [{'dist': 12.0, 'coord': (11, 1)}, {'dist': 11.0, 'coord': (11, 2)}, {'dist': 10.0, 'coord': (11, 3)}, {'dist': 9.0, 'coord': (11, 4)}, {'dist': 8.0, 'coord': (11, 5)}, {'dist': 7.0, 'coord': (11, 6)}, {'dist': 6.0, 'coord': (11, 7)}, {'dist': 5.0, 'coord': (11, 8)}, {'dist': 4.0, 'coord': (11, 9)}, {'dist': 3.0, 'coord': (11, 10)}, {'dist': 2.0, 'coord': (11, 11)}, {'dist': 1.0, 'coord': (11, 12)}]
(11, 12)
angle: 0.08314123188844123, astroids: [{'dist': 12.04, 'coord': (12, 1)}]
(12, 1)
angle: 0.09065988720074511, astroids: [{'dist': 11.05, 'coord': (12, 2)}]
(12, 2)
angle: 0.11065722117389565, astroids: [{'dist': 9.06, 'coord': (12, 4)}]
(12, 4)
angle: 0.12435499454676144, astroids: [{'dist': 8.06, 'coord': (12, 5)}]
(12, 5)
angle: 0.14189705460416394, astroids: [{'dist': 7.07, 'coord': (12, 6)}]
(12, 6)
angle: 0.15264932839526515, astroids: [{'dist': 13.15, 'coord': (13, 0)}]
(13, 0)
angle: 0.16514867741462683, astroids: [{'dist': 12.17, 'coord': (13, 1)}, {'d


----------------------------------------------------------------------
Ran 4 tests in 0.899s

OK


In [2]:
np.pi * 1.5

4.71238898038469

In [3]:
with open("inputs/input_d10.txt") as file:
    map_data = file.read()

am = AstroidMap(map_data)
am.get_coord_with_best_visiblity()

((28, 29), 340)

<h2 id="part2">--- Part Two ---</h2>
<p>Once you give them the coordinates, the Elves quickly deploy an Instant Monitoring Station to the location and discover <span title="The Elves on Ceres just have a unique system of values, that's all.">the worst</span>: there are simply too many asteroids.</p>
<p>The only solution is <em>complete vaporization by giant laser</em>.</p>
<p>Fortunately, in addition to an asteroid scanner, the new monitoring station also comes equipped with a giant rotating laser perfect for vaporizing asteroids. The laser starts by pointing <em>up</em> and always rotates <em>clockwise</em>, vaporizing any
    asteroid it hits.</p>
<p>If multiple asteroids are <em>exactly</em> in line with the station, the laser only has enough power to vaporize <em>one</em> of them before continuing its rotation. In other words, the same asteroids that can be <em>detected</em> can be vaporized, but
    if vaporizing one asteroid makes another one detectable, the newly-detected asteroid won't be vaporized until the laser has returned to the same position by rotating a full 360 degrees.</p>
<p>For example, consider the following map, where the asteroid with the new monitoring station (and laser) is marked <code>X</code>:</p>
<pre><code>.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....X...###..
..#.#.....#....##
</code></pre>
<p>The first nine asteroids to get vaporized, in order, would be:</p>
<pre><code>.#....###<em>2</em><em>4</em>...#..
##...##.<em>1</em><em>3</em>#<em>6</em><em>7</em>..<em>9</em>#
##...#...<em>5</em>.<em>8</em>####.
..#.....X...###..
..#.#.....#....##
</code></pre>
<p>Note that some asteroids (the ones behind the asteroids marked <code>1</code>, <code>5</code>, and <code>7</code>) won't have a chance to be vaporized until the next full rotation. The laser continues rotating; the next nine to be vaporized are:</p>
<pre><code>.#....###.....#..
##...##...#.....#
##...#......<em>1</em><em>2</em><em>3</em><em>4</em>.
..#.....X...<em>5</em>##..
..#.<em>9</em>.....<em>8</em>....<em>7</em><em>6</em>
</code></pre>
<p>The next nine to be vaporized are then:</p>
<pre><code>.<em>8</em>....###.....#..
<em>5</em><em>6</em>...<em>9</em>#...#.....#
<em>3</em><em>4</em>...<em>7</em>...........
..<em>2</em>.....X....##..
..<em>1</em>..............
</code></pre>
<p>Finally, the laser completes its first full rotation (<code>1</code> through <code>3</code>), a second rotation (<code>4</code> through <code>8</code>), and vaporizes the last asteroid (<code>9</code>) partway through its third rotation:</p>
<pre><code>......<em>2</em><em>3</em><em>4</em>.....<em>6</em>..
......<em>1</em>...<em>5</em>.....<em>7</em>
.................
........X....<em>8</em><em>9</em>..
.................
</code></pre>
<p>In the large example above (the one with the best monitoring station location at <code>11,13</code>):</p>
<ul>
    <li>The 1st asteroid to be vaporized is at <code>11,12</code>.</li>
    <li>The 2nd asteroid to be vaporized is at <code>12,1</code>.</li>
    <li>The 3rd asteroid to be vaporized is at <code>12,2</code>.</li>
    <li>The 10th asteroid to be vaporized is at <code>12,8</code>.</li>
    <li>The 20th asteroid to be vaporized is at <code>16,0</code>.</li>
    <li>The 50th asteroid to be vaporized is at <code>16,9</code>.</li>
    <li>The 100th asteroid to be vaporized is at <code>10,16</code>.</li>
    <li>The 199th asteroid to be vaporized is at <code>9,6</code>.</li>
    <li><em>The 200th asteroid to be vaporized is at <code>8,2</code>.</em></li>
    <li>The 201st asteroid to be vaporized is at <code>10,9</code>.</li>
    <li>The 299th and final asteroid to be vaporized is at <code>11,1</code>.</li>
</ul>
<p>The Elves are placing bets on which will be the <em>200th</em> asteroid to be vaporized. Win the bet by determining which asteroid that will be; <em>what do you get if you multiply its X coordinate by <code>100</code> and then add its Y coordinate?</em>    (For example, <code>8,2</code> becomes <em><code>802</code></em>.)</p>

In [4]:
targets_destroyed = am.run_laser()
targets_destroyed[199]

(26, 28)